In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [21]:
# Step 1: Load the pretrained ViT model
model_name = 'google/vit-base-patch16-224-in21k'  # Example pretrained ViT model
model = ViTForImageClassification.from_pretrained(model_name)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Step 2: Modify the model for your classification task
num_classes = 12  # Example: 10 classes in your new classification task
model.classifier = nn.Linear(model.config.hidden_size, num_classes)

In [23]:
# Step 1: Define the path to the root directory containing the train images.
data_dir = "../data/wine_data/images-standard/"


# Step 2: Define data transformations (optional)
# You can apply various transformations like resizing, normalization, or data augmentation.
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to the desired size
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])


# Step 3: Create the custom train dataset using ImageFolder
train_dataset = ImageFolder(root=data_dir, transform=transform)


# Step 4: Create the DataLoader for efficient batch loading
batch_size = 32  # You can adjust this according to your hardware limitations
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


# # Example usage of the DataLoader
# for batch_images, batch_labels in train_loader:
#     # Your training code here
#     pass


In [24]:
# # Step 4: DataLoaders
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64)

In [25]:
# Step 5: Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [28]:
# Step 6: Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

    # model.eval()
    # val_loss = 0.0
    # correct = 0
    # total = 0
    # with torch.no_grad():
    #     for inputs, labels in val_loader:
    #         inputs, labels = inputs.to(device), labels.to(device)


    #         outputs = model(inputs)
    #         _, predicted = torch.max(outputs.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
    #         val_loss += criterion(outputs, labels).item()


    # print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Validation Acc: {100 * correct / total:.2f}%')




RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 19365888 bytes.